In [40]:
import numpy as np
from datetime import datetime
from mmwave import dsp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [41]:
class PointObject:
    def __init__(self, point_cloud,depth,rgb,timestamp):
        self.point_cloud = point_cloud
        self.depth = depth
        self.rgb = rgb
        self.timestamp = timestamp

In [42]:
loaded_depth = np.load("depth_cam.npy",allow_pickle=True)
loaded_depth[0].timestamp

datetime.datetime(2021, 7, 29, 8, 57, 4, 244070)

In [43]:
loaded_mmwave = np.load("radar_cam.npz",allow_pickle=True)
print(loaded_mmwave["arr_0"].shape)
print(loaded_mmwave["arr_1"].shape)

(200, 128, 8, 256)
(200,)


In [44]:
def process_radar(radar_cube): 
    range_resolution = 0.1954
    numTxAntennas = 2
    range_cube = dsp.range_processing(radar_cube) # (num_chirps_per_frame, num_rx_antennas, num_range_bins)
    det_matrix, aoa_input = dsp.doppler_processing(radar_cube)

    numRangeBins = range_cube.shape[2]
    numDopplerBins = aoa_input.shape[2]

    fft2d_sum = det_matrix.astype(np.int64)
    thresholdDoppler, noiseFloorDoppler = np.apply_along_axis(func1d=dsp.ca_,axis=0,arr=fft2d_sum.T,l_bound=1.5,guard_len=4,noise_len=16)

    thresholdRange, noiseFloorRange = np.apply_along_axis(func1d=dsp.ca_,axis=0,arr=fft2d_sum,l_bound=2.5,guard_len=4,noise_len=16)

    thresholdDoppler, noiseFloorDoppler = thresholdDoppler.T, noiseFloorDoppler.T
    det_doppler_mask = (det_matrix > thresholdDoppler)
    det_range_mask = (det_matrix > thresholdRange)

    # # Get indices of detected peaks
    full_mask = (det_doppler_mask & det_range_mask)
    det_peaks_indices = np.argwhere(full_mask == True)

    # # peakVals and SNR calculation
    peakVals = fft2d_sum[det_peaks_indices[:, 0], det_peaks_indices[:, 1]]
    snr = peakVals - noiseFloorRange[det_peaks_indices[:, 0], det_peaks_indices[:, 1]]

    dtype_location = '(' + str(numTxAntennas) + ',)<f4'
    dtype_detObj2D = np.dtype({'names': ['rangeIdx', 'dopplerIdx', 'peakVal', 'location', 'SNR'],
                                'formats': ['<i4', '<i4', '<f4', dtype_location, '<f4']})
    detObj2DRaw = np.zeros((det_peaks_indices.shape[0],), dtype=dtype_detObj2D)
    detObj2DRaw['rangeIdx'] = det_peaks_indices[:, 0].squeeze()
    detObj2DRaw['dopplerIdx'] = det_peaks_indices[:, 1].squeeze()
    detObj2DRaw['peakVal'] = peakVals.flatten()
    detObj2DRaw['SNR'] = snr.flatten()

    # # Further peak pruning. This increases the point cloud density but helps avoid having too many detections around one object.
    detObj2DRaw= dsp.prune_to_peaks(detObj2DRaw, det_matrix, numDopplerBins, reserve_neighbor=True)

    # # --- Peak Grouping
    detObj2D = dsp.peak_grouping_along_doppler(detObj2DRaw, det_matrix, numDopplerBins)
    SNRThresholds2 = np.array([[2, 5], [10, 5], [35, 5]])
    peakValThresholds2 = np.array([[4, 275], [1, 400], [500, 0]])
    detObj2D = dsp.range_based_pruning(detObj2D, SNRThresholds2, peakValThresholds2, numRangeBins, 0.5, range_resolution)

    azimuthInput = aoa_input[detObj2D['rangeIdx'], :, detObj2D['dopplerIdx']]
    x, y, z = dsp.naive_xyz(azimuthInput.T,num_rx=4,num_tx=3)

    xyzVecN = np.zeros((3, x.shape[0]))
    xyzVecN[0] = x *range_resolution * detObj2D['rangeIdx']
    xyzVecN[1] = y *range_resolution * detObj2D['rangeIdx']
    xyzVecN[2] = z *range_resolution * detObj2D['rangeIdx']

    #Apply filter mask
    snr_matrix = detObj2D['SNR'].argsort()[-1024:][::-1]
    snr_matrix.shape

    mask=np.zeros(xyzVecN.shape[1], bool)
    mask[snr_matrix] = True
    filtered_points = xyzVecN[:,mask]
    print(filtered_points.shape)
    reshaped = np.reshape(filtered_points,(-1,32,32))
    print(reshaped.shape)

    return reshaped

# fig = plt.figure(figsize=(20,20))

# ax1 = fig.add_subplot(121, projection='3d')
# #ax2 = fig.add_subplot(122)

# ax1.set_title("3D Scatter Plot")
# ax1.scatter(xyzVecN[0], xyzVecN[1], xyzVecN[2])
# ax1.set_xlabel('Azimuth(cm)')
# ax1.set_ylabel('Range(cm)')
# ax1.set_zlabel('Elevation(cm)')
# # ax1.set_xlim3d(-15, 15)
# # ax1.set_ylim3d(0, 15)
# # ax1.set_zlim3d(-15,15 )

# # ax2.set_title("RGB Frame")
# # ax2.imshow(data['rgb'][i][:, :, ::-1])
# # ax2.set_axis_off()

# plt.subplots_adjust(wspace=0.2)
#plt.savefig('3d_scatter_'+str(i)+'_.png', bbox_inches = "tight",dpi=150)


In [47]:
mm = []
depth = []
gap = np.timedelta64(100, 'ms')
for i in loaded_depth: 
    k = 0
    for j in loaded_mmwave["arr_1"]:
        k = k+1
        if gap > abs(i.timestamp-j):
            print("depth timestamp :%s",i.timestamp)
            print("mmwave timestamp :%s",j)
            try:
                radar_points = process_radar(loaded_mmwave["arr_0"][k])
                mm.append(radar_points)
                depth.append(i.point_cloud)
                print("sucessfully")
            except:
                pass;

arr_1 = np.asarray(mm)
arr_2 = np.asarray(depth)
np.savez("radar_cam_sync_1.npz",arr_1,arr_2)


depth timestamp :%s 2021-07-29 08:57:11.753554
mmwave timestamp :%s 2021-07-29 08:57:11.672772


e:\OneDrive - UNSW\PhD\Experiments\2021-06\openradar\mmwave\dsp\angle_estimation.py:883: RuntimeWarning: invalid value encountered in sqrt
  y_vector = np.sqrt(1 - x_vector ** 2 - z_vector ** 2)


(3, 1024)
(3, 32, 32)
sucessfully
depth timestamp :%s 2021-07-29 08:57:11.753554
mmwave timestamp :%s 2021-07-29 08:57:11.827357
(3, 1024)
(3, 32, 32)
sucessfully
depth timestamp :%s 2021-07-29 08:57:17.559533
mmwave timestamp :%s 2021-07-29 08:57:17.506699
(3, 1024)
(3, 32, 32)
sucessfully
depth timestamp :%s 2021-07-29 08:57:22.512813
mmwave timestamp :%s 2021-07-29 08:57:22.546722
(3, 899)
depth timestamp :%s 2021-07-29 08:57:28.111820
mmwave timestamp :%s 2021-07-29 08:57:28.144696
(3, 962)
depth timestamp :%s 2021-07-29 08:57:33.882370
mmwave timestamp :%s 2021-07-29 08:57:33.824488
(3, 1024)
(3, 32, 32)
sucessfully
depth timestamp :%s 2021-07-29 08:57:39.078471
mmwave timestamp :%s 2021-07-29 08:57:39.026577
(3, 838)


D:\Python\lib\site-packages\ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
